In [2]:
import pandas as pd
import numpy as np

In [3]:
%%time
stocks = pd.read_csv('sp500.csv')

CPU times: total: 15.6 ms
Wall time: 18 ms


In [4]:
print(stocks.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   idnumber             493 non-null    int64  
 1   exchange             493 non-null    object 
 2   symbol               493 non-null    object 
 3   shortname            493 non-null    object 
 4   longname             493 non-null    object 
 5   longnamesort         493 non-null    object 
 6   sector               493 non-null    object 
 7   industry             493 non-null    object 
 8   prices               493 non-null    float64
 9   marketcap            493 non-null    float64
 10  ebitda               463 non-null    float64
 11  revenuegrowth        492 non-null    float64
 12  city                 493 non-null    object 
 13  state                474 non-null    object 
 14  country              493 non-null    object 
 15  fulltimeemployees    486 non-null    flo

In [5]:
stocks.drop(['longnamesort','prices','longbusinesssummary','esgrating','logo'], inplace=True, axis=1)

In [6]:
print(stocks.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   idnumber           493 non-null    int64  
 1   exchange           493 non-null    object 
 2   symbol             493 non-null    object 
 3   shortname          493 non-null    object 
 4   longname           493 non-null    object 
 5   sector             493 non-null    object 
 6   industry           493 non-null    object 
 7   marketcap          493 non-null    float64
 8   ebitda             463 non-null    float64
 9   revenuegrowth      492 non-null    float64
 10  city               493 non-null    object 
 11  state              474 non-null    object 
 12  country            493 non-null    object 
 13  fulltimeemployees  486 non-null    float64
 14  weight             493 non-null    float64
dtypes: float64(5), int64(1), object(9)
memory usage: 57.9+ KB
None
